In [ ]:
#Import relevant modules.
import pygsti
import numpy as _np
from pygsti.algorithms import fiducialselection as FS

import matplotlib.pyplot as plt
%matplotlib inline

import time
import os

basedir = 'D:/measuring/measurement/scripts/Gateset_tomography'

In [ ]:
#Construct a target gateset
gs_target = pygsti.construction.build_gateset([2],[('Q0',)], [u'Gx',u'Gy',u'Gu',u'Gv'], 
                                             [ "X(pi/2,Q0)", "Y(pi/2,Q0)","X(pi,Q0)","Y(pi,Q0)"],
                                             prepLabels=['rho0'], prepExpressions=["0"],
                                             effectLabels=['E0'], effectExpressions=["1"], 
                                             spamdefs={'plus': ('rho0','E0'), 'minus': ('rho0','remainder') } )


In [ ]:
#Let's try to pick out a fiducial set.  

#First, we generate a candidate set which we'll attempt to prune.
#Here, we're looking at all gate string sequences of maximum length 3.

max_length = 2
gates = [u'Gx',u'Gy',u'Gu',u'Gv']#We omit any identity operations here, as we don't want them in our fiducials.

#Important for the minlength arg to equal 0, so we include the empty string.
testFidList = pygsti.construction.list_all_gatestrings(gates,0,max_length)

to_keep_fids = []
for fid in testFidList:
    st=fid.str
    add = True
    firstG = st[:2]
    lastG = st[-2:]
    if (lastG == 'Gx' or lastG == 'Gy' or lastG == 'Gi'):
        add = False
    if add!= False:
        for x in range(len(st)/2-1):
            curG = st[2*x:(2*x+2)]
            nextG = st[2*(x+1):(2*(x+1)+2)]
            if (curG == 'Gx' or curG == 'Gy' or curG == 'Gi') and (nextG == 'Gx' or nextG == 'Gy' or nextG == 'Gi'):
                add = False
                break
    if add:
        to_keep_fids.append(fid)

        
#Compute the preparation fiducials
start = time.time()
prepFidList = FS.optimize_integer_fiducials_slack(gs_target,to_keep_fids,prepOrMeas='prep',\
                                                   fixedNum=6,initialWeights=None,slackFrac=1)
end = time.time()
print
print "Preparation fiducial selection completed in {0} seconds.".format(round(end-start, 7))
print prepFidList

print FS.test_fiducial_list(gs_target,prepFidList,'prep',returnAll=True)

#Compute the measurement fiducials
start = time.time()
measFidList = FS.optimize_integer_fiducials_slack(gs_target,to_keep_fids,prepOrMeas='meas',\
                                                   fixedNum=6,initialWeights=None,slackFrac=1)
end = time.time()
print
print "Measurement fiducial selection completed in {0} seconds.".format(round(end-start, 7))
print measFidList

print FS.test_fiducial_list(gs_target,measFidList,'meas',returnAll=True)


In [ ]:
pygsti.io.write_gatestring_list(os.path.join(basedir,"PrepFiducials.txt"), prepFidList, "My fiducial prep gate strings")
pygsti.io.write_gatestring_list(os.path.join(basedir,"MeasFiducials.txt"), measFidList, "My fiducial meas gate strings")